In [7]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# SudachiPy関連
from sudachipy import tokenizer, dictionary

# SudachiPyのインスタンスを作成
tokenizer_obj = dictionary.Dictionary().create()
split_mode = tokenizer.Tokenizer.SplitMode.A

def sudachi_tokenize(text):
    tokens = []
    morphemes = tokenizer_obj.tokenize(text, split_mode)
    for m in morphemes:
        # 品詞を取得
        pos_info = m.part_of_speech()
        # 例: 名詞/形容詞/動詞(原形)などに限定する
        # Sudachiは品詞名が日本語で返ることがあります。環境に応じてご確認ください。
        # "名詞", "動詞", "形容詞" などを含むかどうか
        if pos_info[0] in ["名詞", "動詞", "形容詞"]:
            # 動詞なら、原形にするなど
            # normalized_form() を取るかどうかは用途次第
            tokens.append(m.normalized_form())
    return tokens

def find_best_k_silhouette(X, k_min=2, k_max=10):
    """
    シルエットスコアを用いて最適なクラスタ数Kを探索する。
    X: TF-IDF行列 (scipy.sparseなど)
    k_min, k_max: K の探索範囲
    return: best_k (シルエットスコアが最大となるK)
    """
    best_score = -1
    best_k = k_min
    
    # k_minからk_maxまで試す
    for k in range(k_min, k_max+1):
        if k >= len(X.toarray()):
            # 文書数よりクラスタ数が多くてもKMeansは動くが、あまり意味がないのでbreakする例
            break

        kmeans = KMeans(n_clusters=k, random_state=42)
        labels = kmeans.fit_predict(X)

        # シルエットスコアを計算 (クラスタ数が1だと計算不可)
        score = silhouette_score(X, labels)
        if score > best_score:
            best_score = score
            best_k = k

    return best_k

def group_and_summarize_dynamic_k(chunks, top_n_words=5, k_min=2, k_max=10):
    """
    SudachiPy で形態素解析 → TF-IDFベクトル化 → KMeans (クラスタ数を自動決定) → 簡易要約
    ---------------------------------------------------------------------------
    Parameters
    ----------
    chunks: list of str
        グルーピング対象となる日本語テキストのリスト
    top_n_words: int
        クラスタを要約する上位キーワードの個数
    k_min, k_max: int
        KMeansのクラスタ数の探索範囲

    Returns
    -------
    best_k: int
        シルエットスコアが最大となったクラスタ数
    cluster_result: dict
        {
            cluster_index: {
                "indices": [chunkのインデックスリスト],
                "keywords": [top_n_wordsのキーワードリスト],
                "representative_chunk": str(代表的な文章)
            },
            ...
        }
    """
    # 1. TF-IDFベクトル化
    vectorizer = TfidfVectorizer(tokenizer=sudachi_tokenize, lowercase=False, min_df=1)
    X = vectorizer.fit_transform(chunks)

    # 2. k_min～k_max の範囲でシルエットスコアを最大化するKを探索
    best_k = find_best_k_silhouette(X, k_min, k_max)

    # 3. 決定した K で再度KMeansクラスタリング
    kmeans = KMeans(n_clusters=best_k, random_state=42)
    labels = kmeans.fit_predict(X)

    # 4. クラスタ中心(centroid)から上位キーワードを抽出
    order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
    terms = vectorizer.get_feature_names_out()

    cluster_result = {}
    for c in range(best_k):
        cluster_indices = np.where(labels == c)[0]

        # 上位キーワード
        top_keywords = [terms[idx] for idx in order_centroids[c, :top_n_words]]

        # 代表文章 (例: クラスタに属する文章の先頭)
        rep_chunk_idx = cluster_indices[0] if len(cluster_indices) > 0 else None
        representative_text = chunks[rep_chunk_idx] if rep_chunk_idx is not None else ""

        cluster_result[c] = {
            "indices": cluster_indices.tolist(),
            "keywords": top_keywords,
            "representative_chunk": representative_text
        }

    return best_k, cluster_result


if __name__ == "__main__":
    # --- サンプル文章 ---
    chunks = [
        "レゾナントデバイスを組み込むことで、工作機械の操作性が向上します。",
        "回転式振動モーターでは実現できなかった多彩な触感を再現します。",
        "弊社ではボイスコイルや電磁石コアを利用したアクチュエータを開発しています。",
        "触覚フィードバックを用いて、ユーザーが機械の状態を直感的に感じることができます。",
        "BLEUやROUGEはテキスト要約や機械翻訳の評価指標として用いられます。",
        "機械共振を利用するため、応答速度が速くリアルな振動を得ることが可能です。",
        "Cosine類似度は文章同士の類似度を測定する際に使われることが多いです。",
        "センサーや駆動IC、ソフトウェアとの組み合わせで高い性能を実現します。"
    ]

    # --- クラスタ数を2~8の範囲で探索する例 ---
    best_k, clustered = group_and_summarize_dynamic_k(chunks, top_n_words=5, k_min=2, k_max=8)

    print(f"最適と判定されたクラスタ数 (best_k): {best_k}\n")
    for cluster_id, info in clustered.items():
        print(f"=== Cluster {cluster_id} ===")
        print(f"Indices: {info['indices']}")
        print(f"Top Keywords: {info['keywords']}")
        print(f"Representative Chunk: {info['representative_chunk']}")
        print()

最適と判定されたクラスタ数 (best_k): 2

=== Cluster 0 ===
Indices: [0, 2, 3, 4, 5, 6]
Top Keywords: ['機械', '為る', 'こと', '用いる', '利用']
Representative Chunk: レゾナントデバイスを組み込むことで、工作機械の操作性が向上します。

=== Cluster 1 ===
Indices: [1, 7]
Top Keywords: ['実現', '式', 'モーター', '再現', '回転']
Representative Chunk: 回転式振動モーターでは実現できなかった多彩な触感を再現します。



/Users/daichisghr/src/sales-support/.venv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [4]:
from openai import OpenAI
import os
from sklearn.cluster import KMeans
import numpy as np
from dotenv import load_dotenv
from sklearn.metrics import silhouette_score

load_dotenv()

# 例: OpenAI APIキーを環境変数から取得
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=OPENAI_API_KEY)

def get_openai_embedding(text, model="text-embedding-ada-002"):
    """
    1つのテキストに対してOpenAI Embeddingsを取得する例。
    """
    response = client.embeddings.create(
        model=model,
        input=text
    )
    embedding = response.data[0].embedding
    return np.array(embedding, dtype=np.float32)

def embed_texts_with_openai(texts):
    """
    一連の文章をOpenAI Embeddingsでベクトル化し、まとめてnumpy配列で返す。
    texts: list of str
    return: np.array of shape (len(texts), embedding_dim)
    """
    embeddings = []
    for txt in texts:
        vec = get_openai_embedding(txt)
        embeddings.append(vec)
    return np.vstack(embeddings)

def find_best_k_silhouette(X, k_min=2, k_max=10):
    """
    シルエットスコアを用いて最適なクラスタ数Kを探索する。
    X: Embedding行列 (num_samples, embedding_dim)
    k_min, k_max: K の探索範囲

    return: best_k (シルエットスコアが最大となるK), best_score
    """
    best_k = k_min
    best_score = -1.0
    
    n_samples = X.shape[0]
    # k_minからk_maxまで試す
    for k in range(k_min, min(k_max, n_samples) + 1):
        # サンプル数より多いクラスタ数でも動くが、あまり意味がないので一応チェック
        if k >= n_samples:
            break
        kmeans = KMeans(n_clusters=k, random_state=42)
        labels = kmeans.fit_predict(X)
        score = silhouette_score(X, labels)
        if score > best_score:
            best_score = score
            best_k = k
    return best_k, best_score

def cluster_texts_openai_dynamic_k(texts, k_min=2, k_max=10):
    """
    OpenAI Embeddingsを使ってテキストをベクトル化し、
    シルエットスコアに基づき最適クラスタ数を動的に決定してからKMeansクラスタリングを行う。
    """
    # 1. Embedding取得
    X = embed_texts_with_openai(texts)

    # 2. 最適Kを決定
    best_k, best_score = find_best_k_silhouette(X, k_min, k_max)
    print(f"最適クラスタ数: {best_k}, シルエットスコア: {best_score:.4f}")
    
    # 3. そのKでクラスタリング
    kmeans = KMeans(n_clusters=best_k, random_state=42)
    labels = kmeans.fit_predict(X)
    
    # 4. クラスタごとに結果をまとめ
    cluster_dict = {}
    for i, label in enumerate(labels):
        cluster_dict.setdefault(label, []).append(texts[i])
    
    return best_k, cluster_dict


if __name__ == "__main__":
    # 例: 対象文章
    chunks = [
        "レゾナントデバイスを取り入れることで、工作機械の操作性が向上します。具体的には、操作者が機械を操作する際の振動や触感を実際に体験することができるため、より直感的な操作が可能となります。また、レゾナントデバイスの応答速度が速いため、リアルタイムでのフィードバックが可能となります。",
        "レゾナントデバイスは、多彩な振動表現や触覚フィードバックが可能で、従来の回転式振動モーターでは実現できなかった、例えば、つるつる、ざらざらといったこすれる感触、水の入ったペットボトルを振る触覚、ボタンを押した感触などを、まるで実際に体験しているかのようなリアルな触り心地を豊かに表現します。これにより、工作機械の操作性やユーザー体験を向上させることが期待できます。",
        "レゾナントデバイスは、ボイスコイルまたは電磁石コアなどを主な動力源とし、機械共振を有効利用した振動デバイスです。それにより、つるつる、ざらざらといたこすれる感触、水の入ったペットボトルを振る触覚、ボタンを押した感触などを、まるで実際に体験しているかのようなリアルな触り心地を豊かに表現します。",
        "**回答2: レゾナントデバイスを使用するためには、センサーによる感知、半導体を使用した駆動IC、それらを制御するソフトウエアなどが必要となります。でも安心してください、我々の技術、製品は全て揃っており、ミネベアミツミの「相合力」で課題を解決することが可能です。",
        "レゾナントデバイスは、ユーザーが「実際にモノに触れているような感覚」を体験できるデバイスで、これを工作機械に適用することで、操作者が直接物理的に触れることなくでも、機械の状態や動きをよりリアルに感じることが可能となります。例えば、工作機械が特定の材料を加工する際の振動や抵抗感を再現することで、操作者は機械の動作状態をより直感的に理解できる可能性があります。",
        "レゾナントデバイスの開発には、磁気回路設計技術や精密加工技術などが必要です。また、センサーによる感知、半導体を使用した駆動IC、それらを制御するソフトウエア等が必要となります。",
        "レゾナントデバイスは応答速度が速く、多彩な振動表現や触覚フィードバックが可能です。従来の回転式振動モーターでは実現できなかった、つるつる、ざらざらといったこすれる感触、水の入ったペットボトルを振る触覚、ボタンを押した感触などを、まるで実際に体験しているかのようなリアルな触り心地を豊かに表現します。",
        "レゾナントデバイスは、高信頼性・ワイドレンジのアクチュエータであり、応答速度が速いため、取り扱いやメンテナンスは比較的容易です。また、弊社では全ての技術、製品が揃っており、センサーによる感知、半導体を使用した駆動IC、それらを制御するソフトウエアなどが必要な場合も、弊社の「相合力」で課題を解決することが可能です。",
        "レゾナントデバイスは、ボイスコイルまたは電磁石コアを主な動力源とし、機械共振を有効利用した振動デバイスです。多彩な振動表現や触覚フィードバックが可能で、例えば、つるつる、ざらざらといったこすれる感触、水の入ったペットボトルを振る触覚、ボタンを押した感触などを、まるで実際に体験しているかのようなリアルな触り心地を豊かに表現します",
        "レゾナントデバイスは豊かな触覚表現能力を有し、新たなユーザー体験を実現させる触覚フィードバックデバイスです。多彩な振動表現や触覚フィードバックが可能で、高信頼性・ワイドレンジのアクチュエーターです。特に、従来の回転式振動モーターでは実現できなかった、つるつる、ざらざらといったこすれる感触、水の入ったペットボトルを振る触覚、ボタンを押した感触などを、まるで実際に体験しているかのようなリアルな触り心地を豊かに表現します。",
        "レゾナントデバイスは触覚フィードバック技術を使用しており、利用者が「実際にモノに触れているような感覚」を体験出来るデバイスを開発しています。主にコイルと磁石を使用し、従来の振動モーターでは成しえないさまざまな振動を作り出しています。",
        "レゾナントデバイスは触覚フィードバック技術を使用して、ユーザーが「実際にモノに触れているような感覚」を体験できます。つるつる、ざらざらといったこすれる感触、水の入ったペットボトルを振る触覚、ボタンを押した感触などを細かく表現することができます",
        "レゾナントデバイスの主な機能は何ですか？",
        "レゾナントデバイスはどのような振動を生成することが可能ですか",
        "レゾナントデバイスの触覚フィードバックはどのように機能しますか？",
        "レゾナントデバイスは、ユーザーが「実際にものに触れているような感覚」を体験できる技術を提供します。工作機械においては、操作者が直接触れる部分に取り付けることで、より直感的な操作感を提供できる可能性があります。また、危険通知や支援機能の開発にも用いられており、工作機械の安全性向上にも寄与できます。",
        "レゾナントデバイスは、ボイスコイルまたは電磁石コアを主な動力源とし、機械共振を利用した振動デバイスで、多彩な振動表現や触覚フィードバックが可能です。これにより、工作機械の操作性を向上させたり、より直感的な操作を可能にすることができます。例えば、工作機械の操作パネルにレゾナントデバイスを組み込むことで、ボタンを押した感触や、材料の硬さなどを触覚で伝えることが可能となります。",
        "レゾナントデバイスは触覚フィードバック技術を使用しており、利用者が「実際にモノに触れているような感覚」を体験出来るデバイスです。また、従来の振動モーターでは成しえないさまざまな振動を作り出します。",
        "レゾナントデバイスは触覚フィードバック技術を提供します。これにより、ユーザーは工作機械を操作する際に、「実際にモノに触れているような感覚」を体験することができます。これは、操作の精度を向上させ、ユーザーの作業効率を改善することにつながります。",
        "レゾナントデバイスは、ボイスコイルや電磁石コアを主な動力源とし、機械共振を有効利用した振動デバイスです。これにより、多彩な振動表現や触覚フィードバックが可能で、応答速度が速く、つるつる、ざらざらといったこすれる感触、水の入ったペットボトルを振る触覚、ボタンを押した感触などを、まるで実際に体験しているかのようなリアルな触り心地を豊かに表現します",
        "レゾナントデバイスは、ボイスコイルや電磁石コアを主な動力源とし、機械共振を有効利用した振動デバイスです。これにより、多彩な振動表現や触覚フィードバックが可能で、応答速度が速く、つるつる、ざらざらといったこすれる感触、水の入ったペットボトルを振る触覚、ボタンを押した感触などを、まるで実際に体験しているかのようなリアルな触り心地を豊かに表現します。",
        "ファナック社からの質問: 社会的課題解決に向けたレゾナントデバイスの活用例はありますか？"
    ]
    
    best_k, clustered = cluster_texts_openai_dynamic_k(chunks, k_min=2, k_max=6)
    
    for c_id, group_texts in clustered.items():
        print(f"--- Cluster {c_id} ---")
        for txt in group_texts:
            print(txt)
        print()

最適クラスタ数: 5, シルエットスコア: 0.1912
--- Cluster 1 ---
レゾナントデバイスを取り入れることで、工作機械の操作性が向上します。具体的には、操作者が機械を操作する際の振動や触感を実際に体験することができるため、より直感的な操作が可能となります。また、レゾナントデバイスの応答速度が速いため、リアルタイムでのフィードバックが可能となります。
レゾナントデバイスは、ユーザーが「実際にモノに触れているような感覚」を体験できるデバイスで、これを工作機械に適用することで、操作者が直接物理的に触れることなくでも、機械の状態や動きをよりリアルに感じることが可能となります。例えば、工作機械が特定の材料を加工する際の振動や抵抗感を再現することで、操作者は機械の動作状態をより直感的に理解できる可能性があります。
レゾナントデバイスは触覚フィードバック技術を使用しており、利用者が「実際にモノに触れているような感覚」を体験出来るデバイスを開発しています。主にコイルと磁石を使用し、従来の振動モーターでは成しえないさまざまな振動を作り出しています。
レゾナントデバイスは触覚フィードバック技術を使用して、ユーザーが「実際にモノに触れているような感覚」を体験できます。つるつる、ざらざらといったこすれる感触、水の入ったペットボトルを振る触覚、ボタンを押した感触などを細かく表現することができます
レゾナントデバイスは、ユーザーが「実際にものに触れているような感覚」を体験できる技術を提供します。工作機械においては、操作者が直接触れる部分に取り付けることで、より直感的な操作感を提供できる可能性があります。また、危険通知や支援機能の開発にも用いられており、工作機械の安全性向上にも寄与できます。
レゾナントデバイスは触覚フィードバック技術を使用しており、利用者が「実際にモノに触れているような感覚」を体験出来るデバイスです。また、従来の振動モーターでは成しえないさまざまな振動を作り出します。
レゾナントデバイスは触覚フィードバック技術を提供します。これにより、ユーザーは工作機械を操作する際に、「実際にモノに触れているような感覚」を体験することができます。これは、操作の精度を向上させ、ユーザーの作業効率を改善することにつながります。

--- Cluster 0 ---
レゾナント

In [9]:
import json
json_file_path = "resonant_device_responses.json"
with open(json_file_path, 'r', encoding='utf-8') as file:
    data_dict = json.load(file)
len(data_dict["responses"])

22

In [ ]:
from openai import OpenAI
import os
import numpy as np
from dotenv import load_dotenv
from sklearn.cluster import DBSCAN
import json

load_dotenv()

json_file_path = "resonant_device_responses.json"
with open(json_file_path, 'r', encoding='utf-8') as file:
    chunks = json.load(file)
    
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

def get_openai_embedding(text, model="text-embedding-ada-002"):
    response = client.embeddings.create(model=model, input=text)
    embedding = response.data[0].embedding
    return np.array(embedding, dtype=np.float32)

def embed_texts_with_openai(texts):
    embeddings = []
    for txt in texts:
        vec = get_openai_embedding(txt)
        embeddings.append(vec)
    return np.vstack(embeddings)

def cluster_texts_dbscan(texts, eps=0.8, min_samples=2):
    """
    OpenAI Embeddingsを使ってテキストをベクトル化し、DBSCANクラスタリングを行う。
    eps, min_samples は適宜チューニングが必要。
    """
    # Embedding取得
    X = embed_texts_with_openai(texts)

    # DBSCAN
    clusterer = DBSCAN(eps=eps, min_samples=min_samples, metric='euclidean')
    labels = clusterer.fit_predict(X)

    # 結果まとめ
    cluster_dict = {}
    for i, label in enumerate(labels):
        # label = -1 はノイズとして扱われる
        cluster_dict.setdefault(label, []).append(texts[i])
    
    return labels, cluster_dict

if __name__ == "__main__":

    labels, clustered = cluster_texts_dbscan(chunks, eps=0.7, min_samples=2)
    
    print("DBSCAN labels:", labels)  # -1 はノイズ
    for c_id, group_texts in clustered.items():
        print(f"--- Cluster {c_id} ---")
        for txt in group_texts:
            print(txt)
        print()

NameError: name 'JSON' is not defined